In [1]:
import folium
from folium.plugins import MarkerCluster

m1 = folium.Map(location=[44, -73], zoom_start=5)

marker_cluster = MarkerCluster().add_to(m1)

folium.Marker(
    location=[40.67, -73.94],
    popup="Add popup text here.",
    icon=folium.Icon(color="green", icon="ok-sign"),
).add_to(marker_cluster)

folium.Marker(
    location=[44.67, -73.94],
    popup="Add popup text here.",
    icon=folium.Icon(color="red", icon="remove-sign"),
).add_to(marker_cluster)

folium.Marker(
    location=[44.67, -71.94],
    popup="Add popup text here.",
    icon=None,
).add_to(marker_cluster)

m1

In [2]:
import numpy as np

size = 100
lons = np.random.randint(-180, 180, size=size)
lats = np.random.randint(-90, 90, size=size)

locations = list(zip(lats, lons))
popups = [f"lon:{lon}<br>lat:{lat}" for (lat, lon) in locations]

Adding all icons in a single call.

In [3]:
icon_create_function = """\
function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-large',
    iconSize: new L.Point(20, 20)
    });
}"""

In [4]:
from folium.plugins import MarkerCluster

m2 = folium.Map(
    location=[np.mean(lats), np.mean(lons)], tiles="Cartodb Positron", zoom_start=2
)

marker_cluster = MarkerCluster(
    locations=locations,
    popups=popups,
    name="1000 clustered icons",
    overlay=True,
    control=True,
    icon_create_function=icon_create_function,
)

marker_cluster.add_to(m2)

folium.LayerControl().add_to(m2)

m2

Explicit `for` loop allows us to customize map parameters.

In [5]:
%%time

m3 = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    tiles='Cartodb Positron',
    zoom_start=2
)

marker_cluster = MarkerCluster(
    name='1000 clustered icons',
    overlay=True,
    control=False,
    icon_create_function=None
)

for k in range(size):
    location = lats[k], lons[k]
    marker = folium.Marker(location=location)
    popup = f'lon:{location[1]}<br>lat:{location[0]}',
    folium.Popup(popup).add_to(marker)
    marker_cluster.add_child(marker)

marker_cluster.add_to(m3)

folium.LayerControl().add_to(m3)

m3

CPU times: total: 15.6 ms
Wall time: 14 ms


`FastMarkerCluster` is not as flexible as `MarkerCluster` but, like the name suggests, it is faster.

In [6]:
from folium.plugins import FastMarkerCluster

In [7]:
%%time
m4 = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    tiles='Cartodb Positron',
    zoom_start=2
)

FastMarkerCluster(data=list(zip(lats, lons))).add_to(m4)

folium.LayerControl().add_to(m4)

m4

CPU times: total: 15.6 ms
Wall time: 7.98 ms


In [8]:
callback = """\
function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({
        icon: "map-marker", markerColor: "red"});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""
m5 = folium.Map(
    location=[np.mean(lats), np.mean(lons)], 
    tiles="Cartodb Positron", 
    zoom_start=2
)

FastMarkerCluster(data=list(zip(lats, lons)), callback=callback).add_to(m5)

m5